In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait as wait
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm

import numpy as np
import pandas as pd
import re
import sys
import time
import requests
import math

In [2]:
# webdriver 자동설정
def set_chrome_driver():
    chrome_options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    return driver

In [3]:
# 리스트를 문자열로 변환하는 함수 (데이터프레임 만들고 나서 사용할 예정)
def list_changer(list):
    st = ''
    for word in list:
        st = st + word + ','
    return st

In [8]:
# 제품별 정보/평점/리뷰 크롤링을 위한, url 패턴 product number 리스트 만들기

list_prod_num = []    # -> 제품별 정보 크롤링 페이지에서 url 패턴에 사용
for page in range(1, 6): #80개기준 5페이지
    url =f'https://smartstore.naver.com/woorisulgallery/category/2a59932df99946be91f799d367bf2f64?st=RECENT&free=false&dt=LIST&subscr=false&page={page}&size=80'
    header = {
         # referer 유저가 이전에 있던 위치
        'referer': 'https://smartstore.naver.com/woorisulgallery/products/7465928721',
        # 내 노트북 정보. Network > Headers의 user_agent 복사
        'user_agent' : 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Mobile Safari/537.36',
    }
    resp = requests.get(url, headers = header)
    html = BeautifulSoup(resp.text,'html.parser')
    print(f'{page}page crwaling')
    
    for item in (html.select('a._1vVKEk_wsi')):
        list_prod_num.append(item['href'][26:])    #url에서 제품별 고유번호 뽑아내고 이거 리스트에 넣어서 패턴에 돌리도록.

    #차단 막는 코드. 랜덤으로 시간차 준다.
    seed = np.random.randint(100)
    np.random.seed(seed)  
    a = np.random.randint(5)
    time.sleep(a)

1page crwaling
2page crwaling
3page crwaling
4page crwaling
5page crwaling


In [9]:
len(list_prod_num)    #총 제품개수 확인

379

# 제품 별 정보, 평점, 리뷰 크롤링

In [13]:
#크롬 창 열기
driver = set_chrome_driver()

prod_info_list = []
review_list = []

for num in tqdm((list_prod_num)):
    try:
        url = f'https://smartstore.naver.com/woorisulgallery/products/{num}'
        driver.get(url)
        time.sleep(5)

        # 제품 정보 뽑아내기
        html = driver.page_source    # page source 얻고
        soup = BeautifulSoup(html, "html.parser")

        prod_info_list.append(soup.find_all('td',class_='ABROiEshTD')[4].text)    # product_name
        prod_info_list.append(soup.find_all('td',class_='ABROiEshTD')[6].text)   # alcholicity(%)
        prod_info_list.append(soup.find_all('td',class_='ABROiEshTD')[8].text)    # capacity(ml)
        prod_info_list.append(int(soup.find('span',class_='_1LY7DqCnwR').text.replace(',','')))    # price
  
        related_tag = soup.find_all('ul',class_='_3Vox1DKZiA')[0].text    # 관련태그들(keywords)
        prod_info_list.append(re.split('#',related_tag[1:]))
       
        # 리뷰 크롤링 시작 (리뷰가 있는 경우만)
        if int(soup.select('strong._2pgHN-ntx6')[0].text)<1:    # 리뷰 1개 이하이면 pass
            pass

        else:
            # 맨처음 리뷰 1페이지 누르고
            wait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#REVIEW > div > div._180GG7_7yx > div.cv6id6JEkg > div > div > a:nth-child(2)'))).click()
            count = 1
            stop = math.ceil(int(soup.select('strong._2pgHN-ntx6')[0].text) / 20)  # 전체 리뷰 수 / 한 페이지 리뷰수 20개 

            while count < stop+1:
                html_review = driver.page_source
                soup_review = BeautifulSoup(html_review, "html.parser")
                review_page = soup_review.find_all('div',class_='_1YShY6EQ56')

                # 해당 페이지의 리뷰 수(len) 만큼 반복
                for index in range(len(review_page)):
                    review_list.append(soup.find_all('td',class_='ABROiEshTD')[4].text)    #product_name 먼저 담고
                    whole = soup.find_all('div',class_='_1YShY6EQ56')[index].text
                    review = re.split('\d\d.\d\d.\d\d.신고', whole)    # 날짜.신고 기준으로 나눔
                    review_list.append(review[0][2:3])    # 평점
                    review_list.append(review[0][3:])    # 작성자 ID
                    review_list.append(re.sub('\n', " ", review[1]))    # 리뷰글 (엔터 \n 띄어쓰기로 변환)

                # 마지막페이지가 아닐 경우 다음페이지 누르기
                if count == stop:
                    break
                else:
                    wait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#REVIEW > div > div._180GG7_7yx > div.cv6id6JEkg > div > div > a.fAUKm1ewwo._2Ar8-aEUTq'))).click()
                    time.sleep(2)
                    count = count + 1
    except:
        print(f'product {num} error. 다음 제품으로 넘어가요')

# 제품정보list dataframe에 담기
df_prod_info = pd.DataFrame(data = np.array(prod_info_list).reshape(-1,5),
                            columns = ['produc_name', 'alcholicity', 'capacity', 'price', 'keywords'])
# keywords 컬럼에 들어간 리스트 껍데기 벗겨내기
df_prod_info['keywords'] = df_prod_info['keywords'].apply(lambda x : list_changer(x))
            
# 리뷰list dataframe에 담기
df_review = pd.DataFrame(data = np.array(review_list).reshape(-1,4),
                         columns = ['product_name', 'score', 'customerID', 'comments'])
print('DONE')

# csv로 저장
df_prod_info.to_csv("woorisool_product_list.csv")
df_review.to_csv("woorisool_review.csv")

  0%|          | 0/379 [00:00<?, ?it/s]

product 4775093943 error. 다음 제품으로 넘어가요
product 4771529448 error. 다음 제품으로 넘어가요


C:\Users\X1Carbon\AppData\Local\Temp\ipykernel_5248\3750459800.py:60: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  df_prod_info = pd.DataFrame(data = np.array(prod_info_list).reshape(-1,5),


DONE
